<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Search-trajectory-files-and-create-output-path" data-toc-modified-id="Search-trajectory-files-and-create-output-path-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Search trajectory files and create output path</a></span></li><li><span><a href="#Plot-experiment-trajectories" data-toc-modified-id="Plot-experiment-trajectories-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Plot experiment trajectories</a></span></li><li><span><a href="#Processing-Experiments" data-toc-modified-id="Processing-Experiments-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Processing Experiments</a></span></li><li><span><a href="#Interactive-plot-of-density-images" data-toc-modified-id="Interactive-plot-of-density-images-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Interactive plot of density images</a></span></li><li><span><a href="#Aggregate-unfiltered-density-distributions" data-toc-modified-id="Aggregate-unfiltered-density-distributions-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Aggregate unfiltered density distributions</a></span></li><li><span><a href="#Filter-duplicated-images" data-toc-modified-id="Filter-duplicated-images-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Filter duplicated images</a></span></li><li><span><a href="#Combine-images" data-toc-modified-id="Combine-images-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Combine images</a></span></li><li><span><a href="#Aggregate-filtered-density-distributions" data-toc-modified-id="Aggregate-filtered-density-distributions-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Aggregate filtered density distributions</a></span></li><li><span><a href="#Filter-50/50-distributions" data-toc-modified-id="Filter-50/50-distributions-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Filter 50/50 distributions</a></span></li><li><span><a href="#Plot-density-maps-to-file" data-toc-modified-id="Plot-density-maps-to-file-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Plot density maps to file</a></span></li></ul></div>

In [1]:
# imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as matplot
import itertools
import math
import sys

from tqdm import tqdm_notebook, tqdm
from os import listdir, mkdir
from os.path import join, isdir
from ipywidgets import *
from multiprocessing import Pool, Lock

module_path = os.path.abspath(os.path.join('../python/python_src_preprocessing/src'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from utils.reader import get_all_trajectory_files, read_trajectory_file

import preprocessing.preprocessing as processing
import plotting.densitymap as plot

%load_ext autoreload
%autoreload 1
%aimport preprocessing.preprocessing, plotting.densitymap

%matplotlib inline

# Search trajectory files and create output path

In [2]:
# paramter
input_path = 'check/outputs/obstacle'
output_path = 'check/csvs/obstacle'
experiments = get_all_trajectory_files(input_path)

if not isdir(output_path):
    mkdir(output_path)

# Plot experiment trajectories

In [3]:
# helper functions

def toInch(pixel, dpi=92):
    return pixel / dpi


def plot_experiment(frame, ax):
    # map p-ids to targets
    for pId, group in frame.groupby('pedestrianId'):
        data = group.loc[(group['x'] > 14.5) & (
            group['x'] < 24.9) & (group['y'] < 8.7)]
        ax.plot(data['x'], data['y'], zorder=1)

    rect = matplot.Rectangle(
        (18.49, 6.69), 2.4, 1, edgecolor='r', alpha=1, fill=False, zorder=2, linewidth=2)

    ax.add_patch(rect)
    ax.grid()
    
def common_start(sa, sb):
    """ returns the longest common substring from the beginning of sa and sb 
    taken from https://stackoverflow.com/questions/18715688/find-common-substring-between-two-strings
    """
    def _iter():
        for a, b in zip(sa, sb):
            if a == b:
                yield a
            else:
                return

    return ''.join(_iter())

In [ ]:
# plot all scenario trajectories to file
# %%capture
plots = len(experiments)
rows = math.ceil(plots / 2.0)
fig, ax = matplot.subplots(nrows=rows, ncols=2, figsize=(
    toInch(1000), toInch(rows * 300)), dpi=92)

ax = np.array(ax).flatten()

for idx in range(plots):
    experiment = experiments[idx]
    directory = os.path.dirname(experiment)
    name = os.path.basename(directory) #.split('_')[0]

    ax[idx].set_title(name)

    frame = read_trajectory_file(experiment)
    plot_experiment(frame, ax[idx])

fig.subplots_adjust(hspace=0.25)
fig.savefig('vadere-scenario-trajectories.pdf', transparent=True)

# Processing Experiments

In [5]:
# processing experiments

results = []
for experiment in tqdm_notebook(experiments, desc='process files'):
    directory = os.path.dirname(experiment)
    name = os.path.basename(directory)

    if os.path.exists(join(output_path, name + '.csv')):
        print('skipping', name)
        continue

    frame = read_trajectory_file(experiment)
    pId2Target = {r['pedestrianId']: r['sourceId'] for i, r in pd.read_csv(
        join(directory, 'sources.csv'), sep=' ', skiprows=1, header=None, names=['pedestrianId', 'sourceId']).iterrows()}
    context = dict({
        'name': name,
        'targets': [1, 2],
        'area':  [18.50, 6.7, 2.4, 1],
        'exact': True,
        'sigma': 0.7,
        'resolution': 0.1,
        'pedestrian_radius': 0.195,
        'gauss_bounds': 2,
        'pId2Target': pId2Target,
        'processors': [
            processing.process_densities,
            processing.process_pedestrians,
            processing.process_percentiles
        ],
        'number_of_cores': 5,
        'skip': 1
    })
    context = processing.process_experiment(frame, context)
    results.append(context)

    densities = np.array(context.get('densities'))
    # sort by timestep number
    densities = densities[densities[:,0].argsort()]
    with open(join(output_path, name + '.csv'), mode='w') as file:
        for density in densities:
            file.write(
                ';'.join(map(lambda r: "{:.10f}".format(r), density[1:])) + '\n')

        file.flush()
        

skipping ORG-KO-240-050-240_2019-06-10_19-37-23.864
using every  1 timestep, # of used timesteps 703 , # of total timesteps 703 image size (10, 24)


100%|████████████████████████████████████████████████████████████████████████████████| 703/703 [00:26<00:00, 12.05it/s]


using every  1 timestep, # of used timesteps 705 , # of total timesteps 705 image size (10, 24)


100%|████████████████████████████████████████████████████████████████████████████████| 705/705 [00:27<00:00, 11.71it/s]


using every  1 timestep, # of used timesteps 666 , # of total timesteps 666 image size (10, 24)


...

100%|████████████████████████████████████████████████████████████████████████████████| 947/947 [01:03<00:00,  5.92it/s]


using every  1 timestep, # of used timesteps 946 , # of total timesteps 946 image size (10, 24)


100%|████████████████████████████████████████████████████████████████████████████████| 946/946 [01:03<00:00,  6.48it/s]


using every  1 timestep, # of used timesteps 919 , # of total timesteps 919 image size (10, 24)


100%|████████████████████████████████████████████████████████████████████████████████| 919/919 [01:00<00:00,  7.37it/s]


# Interactive plot of density images
<font color='red' size='5'><b>Important note:</b><br/>This requires that the result of the processed files (cell above) is still available</font>

In [ ]:
# interactive density maps
%matplotlib notebook

def extrac_xy(pedestrians, x0, y0, resolution):
    x = []
    y = []

    for _, ped in pedestrians.iterrows():
        x.append(int(abs(ped['x'] - x0) * (1 / resolution)))
        y.append(int(abs(ped['y'] - y0) * (1 / resolution)))

    return x, y

for result in results:
    name = result.get('name')
    densities = result.get('densities')
    area = result.get('area')
    peds = result.get('pedestrians')
    size = result.get('size')
    res = result.get('resolution')
    t = len(result.get('targets'))

    start = 0

    first = densities[start][:-t].reshape(size)

    fig, ax = matplot.subplots()

    cbar = None

    ax.set_xticks(np.linspace(0, len(first[0]) - 1, 13))
    ax.set_xticklabels([0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2,
                        1.4, 1.6, 1.8, 2.0, 2.2, 2.4])

    ax.set_yticks(np.linspace(0, len(first) - 1, 6))
    ax.set_yticklabels([0, 0.2, 0.4, 0.6, 0.8, 1.0])

    def update(frame):
        global cbar
        f = densities[frame][:-t].reshape(size)

        ax.clear()
        if cbar is not None:
            cbar.remove()

        img = ax.imshow(f, interpolation='Nearest')
        cbar = fig.colorbar(img)

        x, y = extrac_xy(peds[frame], area[0], area[1], res)

        ax.plot(x, y, '.r', markersize=10)

        ax.set_xticks(np.linspace(0, len(first[0]) - 1, 13))
        ax.set_xticklabels([0, 0.2, 0.4, 0.6, 0.8, 1.0,
                            1.2, 1.4, 1.6, 1.8, 2.0, 2.2, 2.4])

        ax.set_yticks(np.linspace(0, len(first) - 1, 6))
        ax.set_yticklabels([0, 0.2, 0.4, 0.6, 0.8, 1.0])

    interact(update, frame=widgets.IntSlider(
        min=0, max=len(densities)-1, step=1, value=start))

    break

# Aggregate unfiltered density distributions

In [10]:
# aggregate distributions

# output_p = 'juelich-vadere/output_01_save/'
outputs = list(filter(lambda o: os.path.isfile(os.path.join(output_path, o)) and 'filtered' not in o, os.listdir(output_path)))
dfs = []
for output in tqdm_notebook(outputs, desc="Loading density maps", total=len(outputs)):
    dfs.append(pd.read_csv(os.path.join(
        output_path, output), sep=';', header=None))

frame = pd.concat([df.iloc[:, -2:] for df in dfs], sort=False)
print(frame.shape)
display(frame.groupby(list(frame.columns), as_index=False).size().to_frame(
    'frames').reset_index())

(14714, 2)


,240,241,frames
0,0.000000,1.000000,1180
1,0.125000,0.875000,3
2,0.142857,0.857143,10
3,0.166667,0.833333,112
4,0.200000,0.800000,399
5,0.222222,0.777778,2
6,0.250000,0.750000,794
7,0.285714,0.714286,107
8,0.333333,0.666667,1592
9,0.375000,0.625000,41


# Filter duplicated images

In [11]:
# filter consecutive duplicates

outputs = list(filter(lambda o: os.path.isfile(os.path.join(output_path, o)) and 'filtered' not in o, os.listdir(output_path)))

for output in tqdm_notebook(outputs, desc="filter density maps"):
    last = None
    skipped = 0
    written = 0
    with open(os.path.join(output_path, output), "r") as full:
        with open(os.path.join(output_path, output[:-4] + '_filtered.csv'), "w") as filtered:
            for line in full:
                if line != last:
                    filtered.write(line)
                    last = line
                    written = written + 1
                    continue

                skipped = skipped + 1
    print(output, written, 'lines written,', skipped, 'lines skipped')

ORG-KO-240-050-240_2019-06-10_19-37-23.864.csv 247 lines written, 326 lines skipped
ORG-KO-240-050-240_2019-06-28_19-17-42.661.csv 244 lines written, 310 lines skipped
ORG-KO-240-050-240_2019-06-28_19-26-52.509.csv 256 lines written, 299 lines skipped
ORG-KO-240-060-240_2019-06-10_19-38-06.167.csv 229 lines written, 292 lines skipped
ORG-KO-240-060-240_2019-06-28_19-18-23.76.csv 227 lines written, 295 lines skipped
ORG-KO-240-060-240_2019-06-28_19-27-35.805.csv 229 lines written, 319 lines skipped
ORG-KO-240-080-240_2019-06-10_19-38-44.260.csv 385 lines written, 357 lines skipped
ORG-KO-240-080-240_2019-06-28_19-19-01.83.csv 385 lines written, 343 lines skipped
ORG-KO-240-080-240_2019-06-28_19-28-16.408.csv 366 lines written, 359 lines skipped
ORG-KO-240-100-240_2019-06-10_19-39-52.891.csv 323 lines written, 318 lines skipped
ORG-KO-240-100-240_2019-06-28_19-20-11.545.csv 345 lines written, 276 lines skipped
ORG-KO-240-100-240_2019-06-28_19-29-28.361.csv 352 lines written, 281 lines sk

# Combine images

In [ ]:
# combine images of same scenario types

# navigation
outputs = list(filter(lambda o: 'filtered' in o and 'navigation' in o, os.listdir(os.path.join(output_path, 'navigation'))))
pd.concat( [ pd.read_csv(os.path.join(output_path, f)) for f in outputs ], sort=False ).to_csv( os.path.join(output_path, 'navigation', 'combined.csv'), index=False)

# unit
outputs = list(filter(lambda o: 'filtered' in o and 'unit' in o, os.listdir(os.path.join(output_path, 'unit'))))
pd.concat( [ pd.read_csv(os.path.join(output_path, f)) for f in outputs ], sort=False ).to_csv( os.path.join(output_path, 'unit', 'combined.csv'), index=False)

# obstacle weight 0.3
outputs = list(filter(lambda o: ('filtered' in o) and ('navigation' not in o) and ('unit' not in o), os.listdir(os.path.join(output_path, 'obstacle'))))
pd.concat( [ pd.read_csv(os.path.join(output_path, f)) for f in outputs ], sort=False ).to_csv( os.path.join(output_path,'obstacle', 'combined.csv'), index=False)

# Aggregate filtered density distributions

In [12]:
# aggregate distributions

# output_p = 'juelich-vadere/output_01_save/'
# for t in ['navigation', 'unit', 'obstacle']:
outputs = list(filter(lambda o: 'filtered' in o, os.listdir(os.path.join(output_path))))
dfs = []
for output in tqdm_notebook(outputs, desc="Loading density maps", total=len(outputs)):
    dfs.append(pd.read_csv(os.path.join(
        output_path, output), sep=';', header=None))

frame = pd.concat([df.iloc[:, -2:] for df in dfs], sort=False)
print(frame.shape)
frame = frame.groupby(list(frame.columns), as_index=False).size().to_frame(
    'frames').reset_index()

display(frame)

(7895, 2)


,240,241,frames
0,0.000000,1.000000,557
1,0.125000,0.875000,2
2,0.142857,0.857143,7
3,0.166667,0.833333,70
4,0.200000,0.800000,245
5,0.222222,0.777778,1
6,0.250000,0.750000,444
7,0.285714,0.714286,64
8,0.333333,0.666667,853
9,0.375000,0.625000,23


# Filter 50/50 distributions

In [15]:
# filter every second 50% 50% distribution
outputs = list(filter(lambda o: o.endswith('filtered.csv'), os.listdir(output_path)))

for output in tqdm_notebook(outputs, desc="filter density distributions"):
    skip = True
    skipped = 0
    written = 0
    with open(os.path.join(output_path, output), "r") as full:
        with open(os.path.join(output_path, output[:-4] + '_reduced.csv'), "w") as filtered:
            for line in full:
                if line.strip().endswith('0.5000000000;0.5000000000'):
                    skip = not(skip)
                    
                    if skip:
                        skipped = skipped + 1
                        continue
                
                filtered.write(line)
                written = written + 1

    print(written, 'lines written,', skipped, 'lines skipped')
    
# aggregate distributions
outputs = list(filter(lambda o: 'reduced' in o, os.listdir(os.path.join(output_path))))
dfs = []
for output in outputs:
    dfs.append(pd.read_csv(os.path.join(
        output_path, output), sep=';', header=None))

frame = pd.concat([df.iloc[:, -2:] for df in dfs], sort=False)
print(frame.shape)
frame = frame.groupby(list(frame.columns), as_index=False).size().to_frame(
    'frames').reset_index()

display(frame)

213 lines written, 34 lines skipped
219 lines written, 25 lines skipped
224 lines written, 32 lines skipped
202 lines written, 27 lines skipped
201 lines written, 26 lines skipped
203 lines written, 26 lines skipped
337 lines written, 48 lines skipped
344 lines written, 41 lines skipped
324 lines written, 42 lines skipped
286 lines written, 37 lines skipped
309 lines written, 36 lines skipped
308 lines written, 44 lines skipped
432 lines written, 55 lines skipped
423 lines written, 54 lines skipped
433 lines written, 53 lines skipped
450 lines written, 47 lines skipped
427 lines written, 51 lines skipped
434 lines written, 42 lines skipped
414 lines written, 51 lines skipped
424 lines written, 56 lines skipped
416 lines written, 45 lines skipped
(7023, 2)


,240,241,frames
0,0.000000,1.000000,557
1,0.125000,0.875000,2
2,0.142857,0.857143,7
3,0.166667,0.833333,70
4,0.200000,0.800000,245
5,0.222222,0.777778,1
6,0.250000,0.750000,444
7,0.285714,0.714286,64
8,0.333333,0.666667,853
9,0.375000,0.625000,23


# Plot density maps to file

In [7]:
# save density maps

image_directory = "check/images/obstacle"

if not os.path.isdir(image_directory):
    os.mkdir(image_directory)

def filter_chunk(chunk):
    return chunk.loc[(chunk[231] == 0.5) & (chunk[232] == 0.5)]
    
outputs = list(filter(lambda o: 'reduced' in o, os.listdir(output_path)))
for output in tqdm_notebook(outputs):
    plot.processFile(output, output_path, image_directory) # , _filter=filter_chunk

Done ORG-KO-240-050-240_2019-06-10_19-37-23.864_filtered_reduced.csv


Done ORG-KO-240-050-240_2019-06-28_19-17-42.661_filtered_reduced.csv


Done ORG-KO-240-050-240_2019-06-28_19-26-52.509_filtered_reduced.csv


Done ORG-KO-240-060-240_2019-06-10_19-38-06.167_filtered_reduced.csv


Done ORG-KO-240-060-240_2019-06-28_19-18-23.76_filtered_reduced.csv


...

Done ORG-KO-240-150-240_2019-06-28_19-22-59.7_filtered_reduced.csv


Done ORG-KO-240-150-240_2019-06-28_19-32-21.415_filtered_reduced.csv


Done ORG-KO-240-240-240_2019-04-24_17-43-58.231_filtered_reduced.csv


Done ORG-KO-240-240-240_2019-06-28_19-24-47.252_filtered_reduced.csv


Done ORG-KO-240-240-240_2019-06-28_19-34-11.853_filtered_reduced.csv
